In [31]:
#We need to load the pixkled files because we need them for prediction. First import the necessary libraries
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [32]:
#load the ANN trained model, three pickled files: labl_encoder_gender, label_encoder_geo, scaler.pkl#
model = load_model('model.h5')

# #load encoder and scaler
with open('onehot_encoder_gender.pkl', 'rb') as file:
     one_hot_encoder_gen= pickle.load(file)
with open('onehot_encoder_geo.pkl', 'rb') as file:
     one_hot_encoder_geo= pickle.load(file)

# # Load the scaler
with open('scaler.pkl', 'rb') as file:
    scaler = pickle.load(file)

In [33]:
input_data = {
        'CreditScore' : 600, 
        'Geography' : 'France',
        'Gender' : 'Female',
        'Age' : 80,
        'Tenure' : 3,
        'Balance' : 60000,
        'NumOfProducts' : 2, 
        'HasCrCard' :0,
        'IsActiveMember' : 0, 
        'EstimatedSalary': 50000 }

In [34]:

# Convert input_data to DataFrame
input_df = pd.DataFrame([input_data])
input_df

geo_encoder=one_hot_encoder_geo.transform(input_df[['Geography']])
geo_encoder

geo_encoded_df = pd.DataFrame(geo_encoder, columns=one_hot_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df

gen_encoder=one_hot_encoder_gen.transform(input_df[['Gender']])
gen_encoder

gen_encoded_df = pd.DataFrame(gen_encoder, columns=one_hot_encoder_gen.get_feature_names_out(['Gender']))
gen_encoded_df

# Drop the original categorical columns
input_df = input_df.drop(['Geography', 'Gender'], axis=1)


In [35]:
input_df = pd.concat([input_df,gen_encoded_df,geo_encoded_df], axis=1)
input_df

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Gender_Female,Gender_Male,Geography_France,Geography_Germany,Geography_Spain
0,600,80,3,60000,2,0,0,50000,1.0,0.0,1.0,0.0,0.0


In [36]:

# Scale the input features using the loaded scaler
scaled_input = scaler.transform(input_df)
scaled_input


# # Make prediction
prediction = model.predict(scaled_input)
prediction_prob=prediction[0][0]
print(prediction_prob)

if (prediction_prob > 0.5):
   print('The person is likely to churn')
else:
  print('The person is not likely to churn')


1/1 [==============================] - 0s 101ms/step
0.6525171
The person is likely to churn


### Need to create the streamlit app which will be done in app.py
